В данном датасете построим некоторую базовую модель, относительно предсказаний которой можно будет отталкиваться в оценке точности.

Устроим блиц-проверку алгоритмов sklearn по мотивам статьи https://habr.com/ru/post/475552/

По условиям соревнования, критерием оценки будет MAPE. Мы тоже будет его использовать.

In [26]:
import pandas as pd
import numpy as np
from datetime import datetime

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import train_test_split

In [27]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lars
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LarsCV
from sklearn.linear_model import BayesianRidge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import LinearSVR
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
# from matplotlib import pyplot

# Загрузка тренировочного датасета

In [33]:
X = pd.read_csv('EDAv2_Train.zip').drop(columns='price')
X.head(1).T

,0
body_type,седан
brand,AUDI
color,чёрный
fuel_type,бензин
model_year,1990
n_doors,4
production_year,1991
vehicle_transmission,механика
engine_power,174.0
description,"Машина в приличном состоянии ,не гнилая не р..."


In [34]:
y = pd.read_csv('EDAv1_Train.zip', usecols=['price'])
y.head(1)

,price
0,200000.0


In [35]:
X_test = pd.read_csv('EDAv2_Test.zip')
X.head(1).T

,0
body_type,седан
brand,AUDI
color,чёрный
fuel_type,бензин
model_year,1990
n_doors,4
production_year,1991
vehicle_transmission,механика
engine_power,174.0
description,"Машина в приличном состоянии ,не гнилая не р..."


# Проверим готовность данных

In [36]:
# сравним количество брендов в тренировочном и тестовом датасетах
display('Train brands:', X['brand'].nunique())
display('Test brands:', X_test['brand'].nunique())

'Train brands:'

36

'Test brands:'

12

In [37]:
# оставим в тренировочном датасете только те бренды, которые есть в тестовом
brand_list = list(X_test['brand'].unique())
brand_mask = X['brand'].apply(lambda x: True if x in brand_list else False)
X = X[brand_mask]
y = y[brand_mask]
# и проверим это
display('Train brands:', X['brand'].nunique())
display('Test brands:', X_test['brand'].nunique())

'Train brands:'

12

'Test brands:'

12

In [38]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47453 entries, 0 to 86145
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   body_type             47453 non-null  object 
 1   brand                 47453 non-null  object 
 2   color                 47453 non-null  object 
 3   fuel_type             47453 non-null  object 
 4   model_year            47453 non-null  int64  
 5   n_doors               47453 non-null  int64  
 6   production_year       47453 non-null  int64  
 7   vehicle_transmission  47453 non-null  object 
 8   engine_power          47453 non-null  float64
 9   description           45882 non-null  object 
 10  mileage               47453 non-null  int64  
 11  drive_type            47453 non-null  object 
 12  n_owners              47453 non-null  object 
 13  is_original_techpass  47453 non-null  int64  
 14  is_lefthand_drive     47453 non-null  int64  
 15  engine_displacement

В обоих датасетах есть пропуски в объёме двигателя. 
При этом, во время EDA было видно, что у объема высокая корреляция с мощностью двигателя.
Поэтому, в данной ситуации, для baseline удалим этот признак.

Также удалим признак model_year - высокая корреляция с production_year, который д.б. более важным по смыслу.

Признак full_model_name тоже пока что удалим - слишком много вариаций.

Удалим description - ничего хорошего мы с ним здесь не сделаем.

In [39]:
columns_to_drop = ['engine_displacement', 'model_year', 'full_model_name', 'description']
X.drop(columns=columns_to_drop, inplace=True)

In [40]:
# one-hot-encoding для категориальных признаков
X = pd.get_dummies(X)
X.columns

Index(['n_doors', 'production_year', 'engine_power', 'mileage',
       'is_original_techpass', 'is_lefthand_drive', 'body_type_внедорожник',
       'body_type_кабриолет', 'body_type_купе', 'body_type_лимузин',
       'body_type_лифтбек', 'body_type_минивэн', 'body_type_пикап',
       'body_type_родстер', 'body_type_седан', 'body_type_универсал',
       'body_type_фургон', 'body_type_хэтчбек', 'brand_AUDI', 'brand_BMW',
       'brand_HONDA', 'brand_INFINITI', 'brand_LEXUS', 'brand_MERCEDES',
       'brand_MITSUBISHI', 'brand_NISSAN', 'brand_SKODA', 'brand_TOYOTA',
       'brand_VOLKSWAGEN', 'brand_VOLVO', 'color_бежевый', 'color_белый',
       'color_жёлтый', 'color_зелёный', 'color_коричневый', 'color_красный',
       'color_розовый', 'color_серый', 'color_синий', 'color_фиолетовый',
       'color_чёрный', 'fuel_type_бензин', 'fuel_type_газ', 'fuel_type_гибрид',
       'fuel_type_дизель', 'fuel_type_электро', 'vehicle_transmission_автомат',
       'vehicle_transmission_вариатор', 'vehi

# Проверяем модели

## Разделяем на тренировочную и валидационную части

In [41]:
# параметры разделения
test_size = 0.2
seed = 73

In [42]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=test_size, random_state=seed)
print(X_train.shape, y_train.shape)
print(X_valid.shape, y_valid.shape)

(37962, 57) (37962, 1)
(9491, 57) (9491, 1)


In [43]:
# параметры оценки
num_folds = 5
n_iter = 1000
n_estimators = 100
scoring = 'neg_mean_absolute_percentage_error'

In [44]:
# список моделей-кандидатов
# закомментированы совсем плохие по результатам или расходящиеся
models = []
models.append(('LR', LinearRegression(n_jobs=-1)))
models.append(('R', Ridge()))
# models.append(('L', Lasso()))
# models.append(('ELN', ElasticNet()))
# models.append(('LARS', Lars(normalize=False)))
# models.append(('BR', BayesianRidge(n_iter=n_iter)))
models.append(('KNR', KNeighborsRegressor(n_jobs=-1))) # более-менее результат, но хуже ансамблей и очень медленно
models.append(('DTR', DecisionTreeRegressor()))
# models.append(('LSVR', LinearSVR()))
# models.append(('SVR', SVR()))
# models.append(('ABR', AdaBoostRegressor(n_estimators=n_estimators))) # очень плохие результаты
models.append(('BR', BaggingRegressor(n_estimators=n_estimators, n_jobs=-1)))
models.append(('ETR', ExtraTreesRegressor(n_estimators=n_estimators, n_jobs=-1)))
models.append(('GBR', GradientBoostingRegressor(n_estimators=n_estimators)))
models.append(('RFR', RandomForestRegressor(n_estimators=n_estimators, n_jobs=-1)))

In [45]:
# таблица результатов
models_effcy = pd.DataFrame(index=[name for name, model in models])

split_score - чем меньше, тем лучше

cv_mean - чем меньше по модулю, тем лучше

In [46]:
# Оценивание эффективности каждой модели
for name, model in models:
    # оценка с отложенной частью
    m_fit = model.fit(X_train, np.ravel(y_train))
    y_predict = model.predict(X_valid)
    m_score = mean_absolute_percentage_error(y_valid, y_predict)
    models_effcy.loc[name, 'split_score'] = m_score
    # оценка кросс-валидацией по всем данным
    kfold = KFold(n_splits=num_folds, shuffle=True, random_state=seed)
    cv_results = cross_val_score(model, X, np.ravel(y.values), cv=kfold, scoring=scoring)
    models_effcy.loc[name, 'cv_mean'] = cv_results.mean()
    models_effcy.loc[name, 'cv_std'] = cv_results.std()
    # models_effcy.loc[name, 'results'] = list(cv_results)
    print(name, model, 'complete')
display(models_effcy)

LR LinearRegression(n_jobs=-1) complete
R Ridge() complete
KNR KNeighborsRegressor(n_jobs=-1) complete
DTR DecisionTreeRegressor() complete
BR BaggingRegressor(n_estimators=100, n_jobs=-1) complete
ETR ExtraTreesRegressor(n_jobs=-1) complete
GBR GradientBoostingRegressor() complete
RFR RandomForestRegressor(n_jobs=-1) complete


,split_score,cv_mean,cv_std
LR,1.035546,-1.009015,0.016208
R,1.035519,-1.008941,0.016220
KNR,0.435750,-0.429409,0.005950
DTR,0.201309,-0.198031,0.003842
BR,0.160480,-0.158336,0.003721
ETR,0.178764,-0.164117,0.006913
GBR,0.254294,-0.250454,0.004897
RFR,0.162227,-0.158288,0.003805


По показателям снова берем BaggingRegressor для baseline

ТОП для тюнинга и стекинга:
- KNeighborsRegressor
- BaggingRegressor
- RandomForestRegressor
- ExtraTreesRegressor
- DecisionTreeRegressor

# Результат для Kaggle

In [49]:
X_train = pd.read_csv('EDAv2_Train.zip').drop(columns='price')
y_train = pd.read_csv('EDAv2_Train.zip', usecols=['price'])
X_test = pd.read_csv('EDAv2_Test.zip')

In [50]:
# оставим в тренировочном датасете только те бренды, которые есть в тестовом
brand_list = list(X_test['brand'].unique())
brand_mask = X_train['brand'].apply(lambda x: True if x in brand_list else False)
X_train = X_train[brand_mask]
y_train = y_train[brand_mask]

In [51]:
# склеим датасеты признаков
X_train['is_train'] = 1
X_test['is_train'] = 0
X = X_train.append(X_test)

In [52]:
# обработаем также
columns_to_drop = ['engine_displacement', 'model_year', 'full_model_name', 'description']
X.drop(columns=columns_to_drop, inplace=True)
X = pd.get_dummies(X)

In [53]:
# разделим
X_train = X[X['is_train'] == 1].drop(columns=['is_train'])
X_test = X[X['is_train'] == 0].drop(columns=['is_train'])

In [54]:
# обучим
baseline_model = BaggingRegressor(n_estimators=n_estimators, n_jobs=-1)
baseline_model.fit(X_train, np.ravel(y_train))

BaggingRegressor(n_estimators=100, n_jobs=-1)

In [55]:
# предскажем
y_test_predict = baseline_model.predict(X_test)

In [56]:
submission = pd.read_csv('sample_submission_empty.csv')
submission['price'] = y_test_predict
submission.to_csv('sample_submission_baseline_v2.csv', index=False)